<br>

# *Imports* e Funções

In [ ]:
import os
import shutil
import zipfile
import requests
import geopandas as gpd

In [ ]:
os.makedirs(os.path.join('data', 'brutos'), exist_ok=True)

<br>

# Dados Espaciais

## Limites Municipais

In [ ]:
# Define qual o código IBGE do Estado
estado = 'SP'

# Define o nome do arquivo que será salvo as informações do IBGE
file = os.path.join('data', 'brutos', '{}.zip'.format(estado))

# URL para baixar o arquivo zip
url = 'http://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2018/UFs/{}/{}_municipios.zip'.format(estado, str.lower(estado))

# Print
url

In [ ]:
# Define Filename
if url.find('/'):
    filename = url.rsplit('/', 1)[1]
    print(filename)

In [ ]:
# Download
r = requests.get(url, allow_redirects=True)
open(os.path.join('data', 'brutos', filename), 'wb').write(r.content)

In [ ]:
# Unzip
file = os.path.join('data', 'brutos', filename)
temp = os.path.join(os.path.dirname(file), 'temp')
os.makedirs(temp, exist_ok=True)

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall(temp)

In [ ]:
# Lista Arquivos
os.listdir(temp)

In [ ]:
# Read shapefile
gdf = gpd.read_file(os.path.join(temp, '35MUE250GC_SIR.shp'))
display(gdf.head(5))
gdf.plot()

In [ ]:
gdf=gdf[['CD_GEOCMU', 'NM_MUNICIP', 'geometry']]

# Renomeia Colunas
gdf = gdf.rename(
    columns={
        'NM_MUNICIP':'nome_municipio',
        'CD_GEOCMU' :'id_municipio'
    }
)

# Results
print(gdf.dtypes)
display(gdf.head(5))

In [ ]:
# Reprojeta
print(gdf.crs)
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)
gdf.plot()

In [ ]:
# Salva
gdf.to_file(os.path.join('data', 'limite_municipal_sp.geojson'), driver='GeoJSON', encoding='utf-8')
gdf.to_file(os.path.join('data', 'limite_municipal_sp.gpkg'), layer='Limite', driver='GPKG')

In [ ]:
# Excluí pasta temporária
shutil.rmtree(temp)